In [94]:
import pandas as pd
import numpy as np
import re

In [ ]:
url = "https://raw.githubusercontent.com/NicolasCH24/Decks/refs/heads/main/decks.xlsx"

In [95]:
df = pd.read_excel(url)

In [84]:
# INDICA LAS DIMENSIONES DE LA FUENTE DE DATOS EN (FILAS, COLUMNAS)
df.shape

(789, 13)

In [ ]:
# OBTENEMOS EL TIMPO DE DATO POR COLUMNA
df.dtypes

anio                                 float64
expediente                            object
solicitante                           object
tipo_documento                        object
nro_documento                          int64
calle                                 object
altura                                object
barrio                                object
comuna                                 int64
estado                                object
nro_disposicion_resolucion            object
fecha_inicio                  datetime64[ns]
fecha_vencimiento             datetime64[ns]
documento                             object
dtype: object

In [ ]:
# VEMOS LAS COLUMNAS
df.columns

Index(['anio', 'expediente', 'solicitante', 'tipo_documento', 'nro_documento',
       'calle', 'altura', 'barrio', 'comuna', 'estado',
       'nro_disposicion_resolucion', 'fecha_inicio', 'fecha_vencimiento'],
      dtype='object')

In [ ]:
# DESCRIPCION GENERAL DE LA SERIE NUMÉRICA
df.describe()

,anio,nro_documento,comuna,fecha_inicio,fecha_vencimiento
count,788.000000,7.890000e+02,789.000000,789,789
mean,2022.426396,2.729414e+10,11.221800,2023-05-29 04:59:18.935361024,2028-04-23 03:28:03.650190080
min,2019.000000,1.403810e+07,1.000000,2020-10-16 00:00:00,2025-07-21 00:00:00
25%,2022.000000,2.721522e+10,10.000000,2022-09-19 00:00:00,2027-08-08 00:00:00
50%,2023.000000,3.071516e+10,13.000000,2023-07-19 00:00:00,2028-06-07 00:00:00
75%,2023.000000,3.071718e+10,14.000000,2024-01-08 00:00:00,2028-11-30 00:00:00
max,2025.000000,3.371822e+10,15.000000,2025-10-24 00:00:00,2030-07-11 00:00:00
std,1.265680,6.925793e+09,4.297057,NaN,NaN


In [ ]:
# CIUDADANOS UNICOS
df['nro_documento'].nunique()

755

In [ ]:
# OBSERVAMOS DATOS FALTANTES SOLO EN EL AÑO, LO CUAL NO NOS IMPORTA PORQUE NOS VAMOS A BASAR EN LA FECHA DE INICIO DE LA SOLICITUD
df.isnull().any()

anio                           True
expediente                    False
solicitante                   False
tipo_documento                False
nro_documento                 False
calle                         False
altura                        False
barrio                        False
comuna                        False
estado                        False
nro_disposicion_resolucion    False
fecha_inicio                  False
fecha_vencimiento             False
dtype: bool

In [ ]:
# TENEMOS CUITS Y DNIS LOS CAULES DECIDIMOS PASAR A FORMATO DNI
# EL ESTADO UNICO QUE EXISTE ES EL OTORGADO
print(df['tipo_documento'].unique())
print("")
print(df['estado'].unique())

['CUIT' 'DNI']

['OTORGADO']


In [ ]:
# UNA VISTA DE LOS ÚLTIMOS REGISTROS
df.tail(4)

,anio,expediente,solicitante,tipo_documento,nro_documento,calle,altura,barrio,comuna,estado,nro_disposicion_resolucion,fecha_inicio,fecha_vencimiento
785,2019.0,EX-2019-8928504-GCABA-DGOEP,FRESCOLLI Y BATATA S.R.L.,CUIT,30716094894,EL SALVADOR,4676,Palermo,14,OTORGADO,RS-2021-02257974-GCABA-COMUNA14,2021-09-03,2026-09-03
786,2019.0,EX-2019-30635550-GCABA-DGOEP,RUBJOR S.R.L,CUIT,33716296429,ARCOS,1785,Belgrano,13,OTORGADO,RS-2021-9205719-GCABA-COMUNA13,2021-03-19,2026-03-19
787,2019.0,EX-2019-26919035-GCABA-DGOEP,CARECHUR S.R.L,CUIT,33716426829,AMENABAR,2363,Belgrano,13,OTORGADO,RS-2022-36725897-GCABA-COMUNA13,2022-10-13,2027-10-13
788,NaN,EX-2022-30276443- -GCABA-DGCCT- EX-2022-315629...,BUSTOS ROSENDO MATEO,CUIT,20246590797,AGUIRRE,484,Villa Crespo,15,OTORGADO,RS-2023-05998793-GCABA-COMUNA15,2023-02-03,2028-02-03


In [96]:
# NORMALIZAMOS LOS DOCUMENTOS
def get_documento(nro_dni_cuit):
    nro_txt = str(nro_dni_cuit)
    if len(nro_txt) == 11:
        return nro_txt[2:-1]
    elif len(nro_txt) == 8:
        return nro_txt
    else:
        return "Documento erróneo"
    
df['documento'] = df['nro_documento'].apply(get_documento)

In [110]:
def limpiar_altura(valor):
    if pd.isna(valor):
        return np.nan
    valor_str = str(valor)
    # Buscamos el primer número de al menos 3 dígitos
    match = re.search(r'\d{1,}', valor_str)
    if match:
        return int(match.group())
    return np.nan  # si no hay número válido

df['altura_corregida'] = df['altura'].apply(limpiar_altura)
df['altura_corregida'] = df['altura_corregida'].astype(str)

In [111]:
# ORDENAMOS LAS COLUMNAS A GUSTO Y DE MAS ANTIGUA A MAS RECIENTE FECHA DE INICIO
df_ordered = df[['fecha_inicio', 'fecha_vencimiento', 'solicitante', 'documento', 'calle', 'altura_corregida', 'barrio', 'comuna', 'estado', 'expediente', 'nro_disposicion_resolucion']].sort_values(by='fecha_inicio', ascending=True)
df_ordered = df_ordered.rename(columns={'altura_corregida':'altura'})

In [112]:
df_ordered

,fecha_inicio,fecha_vencimiento,solicitante,documento,calle,altura,barrio,comuna,estado,expediente,nro_disposicion_resolucion
765,2020-10-16,2025-10-16,BUENA HUERTA SRL,71609774,CERVIÑO AV.,3889,Palermo,14,OTORGADO,EX-2020-09491688-GCABA-DGCCT,RS-2020-25074837-GCABA-COMUNA14 RS-2022-386534...
782,2020-10-16,2025-10-16,KOI BAR S.R.L,71543324,LAVALLEJA,1387,Palermo,14,OTORGADO,EX-2019-12236586-GCABA-DGOEP,RS-2020-25076833-GCABA-COMUNA14
740,2020-10-19,2025-10-19,BOTAFRIA BRASSERIE S.R.L,71172124,RIVADAVIA AV.,3401,Almagro,5,OTORGADO,EX-2020-21123690- -GCABA-DGCCT,RS-2020-25186983-GCABA-COMUNA5
773,2020-11-09,2025-11-09,PURA MASA S.R.L,71677247,MIÑONES,1886,Belgrano,13,OTORGADO,EX-2020-19425941- -GCABA-DGCDPU,RS-2020-27123653-GCABA-COMUNA13
756,2020-11-10,2025-11-10,PAMPANITO S.A,71567564,"RIVADAVIA MARTIN, COMODORO",1694,Núñez,13,OTORGADO,EX-2020-22334563- -GCABA-DGCCT,RS-2020-27311791-GCABA-COMUNA13
...,...,...,...,...,...,...,...,...,...,...,...
5,2025-06-27,2030-06-27,MATEUS MARTINEZ ANGELICA MARIA,95569901,LAPRIDA,1344,Recoleta,2,OTORGADO,EX-2025-08244233- -GCABA-DGPF,DI-2025-1858-GCABA-DGPF
103,2025-06-30,2030-06-30,REENCUENTRO 6401 SRL.,71773059,DORREGO AV.,1194,Chacarita,15,OTORGADO,EX-2024-48524971- -GCABA-DGPF,DI-2025-1924-GCABA-DGPF
81,2025-07-11,2030-07-11,Cooperativa de Trabajo Centro Cultural Nueva U...,71790979,URIARTE,1289,Palermo,14,OTORGADO,EX-2024-41678969- -GCABA-DGPF,DI-2025-2044-GCABA-DGPF\n
1,2025-07-21,2025-07-21,IGNACIO ROBERTO GOMEZ,39626712,ARENALES,2434,Recoleta,2,OTORGADO,EX-2025-13053524- -GCABA-DGPF,DI-2025-2142-GCABA-DGPF


In [113]:
# RESUMEN DE DATOS A INGRESAR EN LA TABLA
print("**RESUMEN DE DATOS**")
print(f"Único estado: {df_ordered['estado'].unique()[0]}")
print("")
print(f"Primera fecha de inicio de solicitud {df_ordered['fecha_inicio'].min()}")
print(f"Última fecha de inicio de solicitud {df_ordered['fecha_inicio'].max()}")
print("")
print(f"Solicitantes únicos {df_ordered['solicitante'].nunique()}")
print("")
print("Solicitantes por comuna:")
print(df_ordered.groupby('comuna')['solicitante'].nunique().sort_values(ascending=False))
print("")
print("Solicitudes por barrio:")
print(df_ordered.groupby('barrio')['solicitante'].nunique().sort_values(ascending=False))

**RESUMEN DE DATOS**
Único estado: OTORGADO

Primera fecha de inicio de solicitud 2020-10-16 00:00:00
Última fecha de inicio de solicitud 2025-10-24 00:00:00

Solicitantes únicos 776

Solicitantes por comuna:
comuna
14    301
13    137
15     67
2      63
11     43
12     36
6      33
10     23
1      19
4      18
5      18
7      14
3       9
9       4
Name: solicitante, dtype: int64

Solicitudes por barrio:
barrio
Palermo              301
Belgrano              89
Recoleta              63
Villa Crespo          45
Caballito             33
Villa Devoto          25
Núñez                 24
Colegiales            24
Villa Urquiza         19
Chacarita             18
Almagro               14
Villa del Parque      14
Saavedra              13
La Boca               12
Villa Luro            11
San Telmo             11
Flores                 9
Puerto Madero          5
Barracas               5
Balvanera              5
Parque Chacabuco       5
Floresta               5
Monte Castro           4
Boedo

In [114]:
# TABLA A EXPORTAR EN EL DATAWAREHOUSE - DIM_TMP_DECKS
df_ordered['año'] = df['fecha_inicio'].dt.year

In [115]:
df_ordered.dtypes

fecha_inicio                  datetime64[ns]
fecha_vencimiento             datetime64[ns]
solicitante                           object
documento                             object
calle                                 object
altura                                object
barrio                                object
comuna                                 int64
estado                                object
expediente                            object
nro_disposicion_resolucion            object
año                                    int32
dtype: object

## **CONECTO Y CONSULTOR**

In [116]:
"""Módulo para la gestión de conexiones a bases de datos.

Este módulo proporciona funcionalidades para establecer conexiones seguras
a bases de datos Oracle utilizando variables de entorno para la configuración.

Class
---------
ConnectorBACGC_PRD
    Módulo para conectar y realizar consultas al DW de GC_PRD.
"""
import oracledb
from sqlalchemy import create_engine, text
from contextlib import contextmanager
from sqlalchemy.orm import sessionmaker
from sys import modules

GC_DEV = "GC_DEV"
GC_DEV_PASS = "EGvQMVS3"
GC_DEV_DSN = f"{"bicdb-scan.gcba.gob.ar"}/{"sasdev.gcba.gob.ar"}"


class ConnectorBACGC_DEV:
    """
    Clase para gestionar la conexión al DW para el esquema de GC_PRD.
    
    """
    
    def __init__(self):
        """
        Inicializa la clase con las credenciales de conexión a la base de datos.
        """
        self.GC_PRD  = GC_DEV
        self.GC_PASS = GC_DEV_PASS
        self.GC_DSN  = GC_DEV_DSN

    def engine_gc_prd(self):
        """
        Crea y retorna un motor de conexión a la base de datos Oracle.

        Returns
        -------
        sqlalchemy.engine.Engine
            Motor de conexión configurado para la base de datos Oracle.
        """
        modules["cx_Oracle"] = oracledb
        engine = create_engine(
            'oracle+oracledb://',
            connect_args = {
                'user': self.GC_PRD,
                'password': self.GC_PASS, 
                'dsn': self.GC_DSN},
            isolation_level="READ COMMITTED"
            )
        return engine
        
    @contextmanager
    def db_session(self):
        """
        Contexto para manejar sesiones de base de datos.

        Yields
        ------
        sqlalchemy.orm.Session
            Sesión de base de datos activa.

        Notes
        -----
        Esta función maneja automáticamente el commit en caso de éxito
        y el rollback en caso de error.
        """
        engine = self.engine_gc_prd()
        Session = sessionmaker(bind = engine)
        session = Session()
        try:
            print("Sesión de base de datos establecida...")
            yield session
            session.commit()
        except Exception as e:
            session.rollback()
            print(f"Error en la consulta: {e}")
            raise e
        finally:
            print("Cerrando sesión de base de datos...")
            session.close()

In [117]:
conector = ConnectorBACGC_DEV()

In [ ]:
# PRIMERO CREAMOS LA TABLA
query = text("""
CREATE TABLE GC_DEV.DECKS (
    FECHA_INICIO               DATE,
    FECHA_VENCIMIENTO          DATE,
    SOLICITANTE                VARCHAR2(200),
    DOCUMENTO                  VARCHAR2(20),
    CALLE                      VARCHAR2(200),
    ALTURA                     VARCHAR2(20),
    BARRIO                     VARCHAR2(200),
    COMUNA                     NUMBER,
    ESTADO                     VARCHAR2(50),
    EXPEDIENTE                 VARCHAR2(100),
    NRO_DISPOSICION_RESOLUCION VARCHAR2(200),
    AÑO                        NUMBER
)
""")
try:
    with ConnectorBACGC_DEV().db_session() as s:
        s.execute(query)
except Exception as e:
    print(e)

Sesión de base de datos establecida...
Cerrando sesión de base de datos...


In [118]:
# DIVIDIMOS EN DOS PARA HACER LA PRIMERA CARGA Y LUEGO UN UPDATE ROWS CON LA PARTE DOS
df1 = df_ordered[df_ordered['año'] <= 2024]
df2 = df_ordered[df_ordered['año'] > 2024]

In [119]:
def get_max_fecha_inicio(connector):
    query = text("SELECT MAX(FECHA_INICIO) AS max_fecha FROM GC_DEV.DECKS")
    
    with connector.db_session() as s:
        result = s.execute(query).fetchone()
        return result[0] if result[0] is not None else None
    

def filtrar_nuevos(df, max_fecha):
    if max_fecha is None:
        return df
    
    df_nuevo = df[df["fecha_inicio"] > max_fecha].copy()
    return df_nuevo

In [121]:
def insert_new_data(df, conector):
    insert_query = text("""
INSERT INTO GC_DEV.DECKS (
    FECHA_INICIO,
    FECHA_VENCIMIENTO,
    SOLICITANTE,
    DOCUMENTO,
    CALLE,
    ALTURA,
    BARRIO,
    COMUNA,
    ESTADO,
    EXPEDIENTE,
    NRO_DISPOSICION_RESOLUCION,
    AÑO
)
VALUES (
    :fecha_inicio,
    :fecha_vencimiento,
    :solicitante,
    :documento,
    :calle,
    :altura,
    :barrio,
    :comuna,
    :estado,
    :expediente,
    :nro_disposicion_resolucion,
    :año
)
""")
    try:
        with conector.db_session() as s:
            for _, row in df.iterrows():
                s.execute(
                    insert_query,
                    {
                        "fecha_inicio": row["fecha_inicio"],
                        "fecha_vencimiento": row["fecha_vencimiento"],
                        "solicitante": row["solicitante"],
                        "documento": row["documento"],
                        "calle": row["calle"],
                        "altura": row["altura"],
                        "barrio": row["barrio"],
                        "comuna": int(row["comuna"]) if not pd.isna(row["comuna"]) else None,
                        "estado": row["estado"],
                        "expediente": row["expediente"],
                        "nro_disposicion_resolucion": row["nro_disposicion_resolucion"],
                        "año":int(row["año"]) if not pd.isna(row["año"]) else None
                    }
                )
        return "Nuevos registros insertados"
    
    except Exception as e:
        return f"Error al insertar datos nuevos: {e}"

In [ ]:
max_fecha_bd = get_max_fecha_inicio(conector)
df_nuevos = filtrar_nuevos(df1, max_fecha_bd)

if len(df_nuevos) > 0:
    return_insert = insert_new_data(df_nuevos, conector)
    print(return_insert)

Sesión de base de datos establecida...
Cerrando sesión de base de datos...
